The Australian DEM is 35GB, and hosted by the DEA [here](https://data.dea.ga.gov.au/?prefix=projects/elevation/ga_srtm_dem1sv1_0/)
AWS bucket name to use to access the data is `dea-public-data`

## Original Data Source

*SRTM-derived 1 Second Digital Elevation Models Version 1.0*

The 1 second Shuttle Radar Topography Mission (SRTM) Digital Elevation
Models Version 1.0 package comprises three surface models: the Digital
Elevation Model (DEM), the Smoothed Digital Elevation Model (DEM-S)
and the Hydrologically Enforced Digital Elevation Model (DEM-H).

### sources and guides
[DEA guide to accessing data from their public S3 bucket using STAC](https://docs.dea.ga.gov.au/guides/setup/gis/stac/)

[Downloading and streaming data using STAC metadata](https://docs.dea.ga.gov.au/notebooks/How_to_guides/Downloading_data_with_STAC/)

In [ ]:
import os
import numpy

import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt

import geopandas as gpd
from shapely.ops import transform
import rasterio
from rasterio.windows import Window, from_bounds
from rasterio import plot

import pystac_client
import odc.stac
from odc.algo import to_rgba
import stackstac
import pyproj

import leafmap.leafmap as leafmap

import xarray as xr
import localtileserver

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = 'proxy/{port}'

In [ ]:
# #leaving this alone for now as I don't get how to make it interactive. Will use hardcoded file as palceholder.
# uploader = widgets.FileUpload(
#     accept='.geojson',
#     multiple=False
# )
# display(uploader)

In [ ]:
# set variables here
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

geom_gpd = gpd.read_file('data/temp/dissolved-boundaries.geojson')
# for some reason these don't work for leafmap, but the hardcoded values do. Convert to float32?
geom_x = geom_gpd.centroid.x
geom_y = geom_gpd.centroid.y
bbox = list(geom_gpd.total_bounds)

catalog_dem = pystac_client.Client.open("https://explorer.sandbox.dea.ga.gov.au/stac/")
catalog_rgb = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")

collection_dem = ['ga_srtm_dem1sv1_0']
collection_rgb = ['sentinel-2-l2a']

# start and end date for landsat imagery. This is for making a colour image to compare the DEM to.
start_date = "2023-01-01"
end_date = "2023-02-01"

# spatial projection information
crs_string = "epsg:3857"
epsg = pyproj.CRS.from_string(crs_string).to_epsg()
resolution = 90 #added to resolve error "Failed to auto-guess CRS/resolution."

print(bbox)
print(geom_x)
print(geom_y)

In [ ]:
# Build a query with the set parameters
query_dem = catalog_dem.search(
    bbox=bbox, 
    collections=collection_dem
)

# Search the STAC catalog for all items matching the query
items_dem = list(query_dem.items())
print(f"Found: {len(items_dem):d} datasets")

In [ ]:
ds_dem = odc.stac.stac_load(
    items_dem,
    bands=("dem_h"),
    crs=crs_string,
    resolution = resolution,
    chunks={},
    groupby="solar_day",
    bbox=bbox,
)
ds_dem

In [ ]:
ds_rgb = catalog_rgb.search(bbox=bbox,
                             datetime=f"{start_date}/{end_date}",
                             collections=collection_rgb).item_collection()


len(ds_rgb)

In [ ]:
rgb_stack = stackstac.stack(items=ds_rgb,
                            epsg=epsg,
                            bounds_latlon=bbox,
                            assets=["blue", "green", "red"]) #bounds_latlong clips the data within the given area
rgb_stack

In [ ]:
data = rgb_stack.resample(time="MS").median("time", keep_attrs=True)
data


One thing to watch out for with stackstac.stack is that you will wind up with a distinct time coordinate for each STAC item that you pass in. To achieve the intuitive representation of the data, you need to flatten the DataArray with respect to day.
Note: if you are only reading a single STAC item, stackstac.mosaic will inadvertently reduce your data along the band dimension (which is definitely not what you want!), hence the conditional statement checking for more than one time coordinate value.
[from here](https://hrodmn.dev/posts/stackstac/)

In [ ]:
def flatten(x, dim="time"):
    assert isinstance(x, xr.DataArray)
    if len(x[dim].values) > len(set(x[dim].values)):
        x = x.groupby(dim).map(stackstac.mosaic)

    return x


# round time coordinates so all observations from the same day so they have
# equivalent timestamps
data = data.assign_coords(
    time=data.time.astype("datetime64[D]")
)

# mosaic along time dimension
flat_stack = flatten(data, dim="time")
flat_stack

In [ ]:
# flat_stack = flat_stack.compute()

# flat_stack.sel(band=["red", "green", "blue"]).plot.imshow(
#     col="time",
#     col_wrap=4,
#     rgb="band",
#     robust=True,
#     size=6,
#     vmin=0,
#     vmax=3000,
#     add_labels=False,
# )

In [ ]:
ds_dem.dem_h.plot()

In [ ]:
# Initialize a leafmap map
m = leafmap.Map(height="600px", width="800px", draw_control=False, measure_control=False, fullscreen_control=False, attribution_control=False, center=[-29.26, 116.32], zoom=12)

m.add_gdf(geom_gpd, layer_name="boundary", info_mode=None, show=True,style={
    "stroke": True,
    "color": "black",
    "weight": 3,
    "opacity": 1,
    "fill": False})

m.add_raster(ds_dem)
m